In [21]:
import pygal
import pandas as pd
import streamlit as st
from datetime import datetime

In [22]:
# Definindo a data mínima e máxima
data_minima = pd.to_datetime('2024-06-02')
data_maxima = pd.to_datetime('2024-06-08')

    
# Adicionar seletor de períodos na coluna à esquerda
start_date_ = st.sidebar.date_input("Data de início", data_minima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #"DD-MM-YYYY"

end_date_ = st.sidebar.date_input("Data de término", data_maxima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #+ pd.DateOffset(days=1) #"DD-MM-YYYY"

start_date = start_date_.strftime('%Y-%m-%d') #'%d-%m-%Y'
end_date = end_date_.strftime('%Y-%m-%d')

periodo = end_date_ - start_date_

# Periodo anteior para gráficos com comparações
start_date_b4 = start_date_ - pd.DateOffset(days = periodo.days)

end_date_b4 = end_date_ - pd.DateOffset(days = periodo.days)

In [23]:
import csv

def encontrar_frase_em_csv_meta(nome_arquivo, frase_procurada):
    try:
        with open(nome_arquivo, 'r', newline='', encoding='utf-16') as arquivo_csv:
            leitor_csv = csv.reader(arquivo_csv)
            
            for numero_linha, linha in enumerate(leitor_csv, start=1):
                if frase_procurada in linha:
                    return numero_linha

        # Se a frase não for encontrada em nenhuma linha
        return -1

    except FileNotFoundError:
        print(f'O arquivo {nome_arquivo} não foi encontrado.')
        return -1

In [24]:
inicio_alcanceIG = encontrar_frase_em_csv_meta('tabelas/meta/Alcance.csv', 'Alcance do Instagram')

alcanceIG = pd.read_csv('tabelas/meta/Alcance.csv', skiprows=inicio_alcanceIG, encoding='utf-16',skip_blank_lines=True)

inicio_visitasIG = encontrar_frase_em_csv_meta('tabelas/meta/Visitas.csv', 'Visitas ao perfil do Instagram')

visitasIG = pd.read_csv('tabelas/meta/Visitas.csv', skiprows=inicio_visitasIG, encoding='utf-16',skip_blank_lines=True)

inicio_seguidoresIG = encontrar_frase_em_csv_meta('tabelas/meta/Seguidores.csv', 'Seguidores no Instagram')

seguidoresIG = pd.read_csv('tabelas/meta/Seguidores.csv', skiprows=inicio_seguidoresIG, encoding='utf-16',skip_blank_lines=True)


In [25]:
def transformaData_final(data):
    return (pd.to_datetime(data)-pd.DateOffset(days=1)).strftime('%d-%m-%Y')

def transformaData_inicio(data):
    return (pd.to_datetime(data)).strftime('%d-%m-%Y')

In [26]:

import plotly.graph_objects as go

In [28]:
#FILTRO ALCANCE
dados_IG_alcance_FILTRADAS = alcanceIG.copy()

dados_IG_alcance_FILTRADAS = dados_IG_alcance_FILTRADAS.loc[(dados_IG_alcance_FILTRADAS['Data']>start_date) & (dados_IG_alcance_FILTRADAS['Data']<end_date)]

dados_IG_alcance_ANTERIOR = alcanceIG.copy()

dados_IG_alcance_ANTERIOR = dados_IG_alcance_ANTERIOR.loc[(dados_IG_alcance_ANTERIOR['Data']>start_date_b4.strftime('%Y-%m-%d')) & (dados_IG_alcance_ANTERIOR['Data']<end_date_b4.strftime('%Y-%m-%d'))]

#FILTRO VISITAS
visitasIG_FILTRADO = visitasIG.copy()

visitasIG_FILTRADO = visitasIG_FILTRADO.loc[(visitasIG_FILTRADO['Data']>start_date) & (visitasIG_FILTRADO['Data']<end_date)]

visitasIG_ANTERIOR = visitasIG.copy()

visitasIG_ANTERIOR = visitasIG_ANTERIOR.loc[(visitasIG_ANTERIOR['Data']>start_date_b4.strftime('%Y-%m-%d')) & (visitasIG_ANTERIOR['Data']<end_date_b4.strftime('%Y-%m-%d'))]

#FILTRO SEGUIDOORES
seguidoresIG_FILTRADO = seguidoresIG.copy()

seguidoresIG_FILTRADO = seguidoresIG_FILTRADO.loc[(seguidoresIG_FILTRADO['Data']>start_date) & (seguidoresIG_FILTRADO['Data']<end_date)]

seguidoresIG_ANTERIOR = seguidoresIG.copy()

seguidoresIG_ANTERIOR = seguidoresIG_ANTERIOR.loc[(seguidoresIG_ANTERIOR['Data']>start_date_b4.strftime('%Y-%m-%d')) & (seguidoresIG_ANTERIOR['Data']<end_date_b4.strftime('%Y-%m-%d'))]

In [29]:

    # Supor que 'Data' é a coluna de datas e que estas estão no tipo datetime
    visitasIG_FILTRADO['Data'] = pd.to_datetime(visitasIG_FILTRADO['Data'])
    visitasIG_ANTERIOR['Data'] = pd.to_datetime(visitasIG_ANTERIOR['Data'])

    # Manter as datas originais
    visitasIG_FILTRADO['Data_Original'] = visitasIG_FILTRADO['Data']
    visitasIG_ANTERIOR['Data_Original'] = visitasIG_ANTERIOR['Data']

    # Calcular a diferença de dias entre o início dos dois períodos
    delta = visitasIG_FILTRADO['Data'].min() - visitasIG_ANTERIOR['Data'].min()

    # Ajustar as datas do período anterior para sobrepor ao período atual
    visitasIG_ANTERIOR['Data'] = visitasIG_ANTERIOR['Data'] + delta

    # Criar a coluna de sequência de números
    visitasIG_FILTRADO['Dia'] = range(1, len(visitasIG_FILTRADO) + 1)
    visitasIG_ANTERIOR['Dia'] = range(1, len(visitasIG_ANTERIOR) + 1)

    # Cria a figura
    fig = go.Figure()

    # Adiciona a linha do período atual
    fig.add_trace(go.Scatter(
    x=visitasIG_FILTRADO['Dia'],
    y=visitasIG_FILTRADO['Primary'],
    mode='lines+markers',
    name=f'Selecionado: {transformaData_inicio(start_date)} a {transformaData_final(end_date)}',
    text=visitasIG_FILTRADO['Data_Original'].dt.strftime('%Y-%m-%d'),
    hovertemplate='<br>Data Original: %{text}<br>Valor: %{y}',
    line=dict(color='#E1306C')
    ))

    # Adiciona a linha do período anterior
    fig.add_trace(go.Scatter(
    x=visitasIG_ANTERIOR['Dia'],
    y=visitasIG_ANTERIOR['Primary'],
    mode='lines+markers',
    name=f"Anterior: {start_date_b4.strftime('%d-%m-%Y')} a {(end_date_b4 - pd.DateOffset(days=1)).strftime('%d-%m-%Y')}",
    text=visitasIG_ANTERIOR['Data_Original'].dt.strftime('%d-%m-%Y'),
    hovertemplate='<br>Data Original: %{text}<br>Valor: %{y}',
    line=dict(color='#EB7099')
    ))

    # Configura o layout
    fig.update_layout(
    title='Visitas do IG - Comparação de Períodos',
    title_x=0.1,
    title_xanchor='left',
    title_y=0.85,
    xaxis_title='Dias',
    yaxis_title='Alcance',
    hovermode='x unified',
    yaxis=dict(tickformat='.'),
    legend=dict(x=1, y=1, xanchor='right', yanchor='bottom', font=dict(size=10))
    )
    fig.show()